# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import json
import requests
import csv
import numpy as np
import pandas as pd
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.automl.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-courtlin",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano-degree")
exp = Experiment(workspace=ws, name="stroke-prediction")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-courtlin
Azure region: eastus2
Subscription id: bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8
Resource group: udacity-nano-degree


In [3]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [4]:
# Create the CPU computer cluster
amlcompute_cluster_name = "capstone"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=5,
                                                           min_nodes=0,
                                                           idle_seconds_before_scaledown=600)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-courtlin",subscription_id="bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8", resource_group="udacity-nano-degree")
experiment = Experiment(workspace=ws, name="hyperdrive-ml")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: udacity-courtlin
Azure region: eastus2
Subscription id: bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8
Resource group: udacity-nano-degree


In [6]:
# ws = Workspace.from_config()
# experiment_name = 'hyperdrive-ml'

# experiment=Experiment(ws, experiment_name)

In [7]:
#Access the external data needed for the project that has been downloaded from Kaggle and uploaded to the Azure workspace
#This dataset, title Stroke Prediction Dataset, was downloaded from Kaggle for educational use. URL: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset?select=healthcare-dataset-stroke-data.csv
#The author of the dataset is fedesoriano

# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='healthcare-dataset-stroke-data')

# from azureml.data.dataset_factory import TabularDatasetFactory
# from azureml.core.dataset import Dataset

# url_paths = [
#     "https://www.kaggle.com/fedesoriano/stroke-prediction-dataset/download"
#     ]

# # dataset = TabularDatasetFactory.from_delimited_files(path=url_paths)
# dataset = Dataset.Tabular.from_delimited_files(path=url_paths)
# type(dataset)

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
gender               5110 non-null object
age                  5110 non-null float64
hypertension         5110 non-null bool
heart_disease        5110 non-null bool
ever_married         5110 non-null bool
work_type            5110 non-null object
Residence_type       5110 non-null object
avg_glucose_level    5110 non-null float64
bmi                  4909 non-null float64
smoking_status       5110 non-null object
stroke               5110 non-null bool
dtypes: bool(4), float64(3), object(4)
memory usage: 299.5+ KB


In [8]:
df.describe()

,age,avg_glucose_level,bmi
count,5110.000000,5110.000000,4909.000000
mean,43.226614,106.147677,28.893237
std,22.612647,45.283560,7.854067
min,0.080000,55.120000,10.300000
25%,25.000000,77.245000,23.500000
50%,45.000000,91.885000,28.100000
75%,61.000000,114.090000,33.100000
max,82.000000,271.740000,97.600000


In [9]:
df.head(10)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,False,True,True,Private,Urban,228.69,36.6,formerly smoked,True
1,Female,61.0,False,False,True,Self-employed,Rural,202.21,NaN,never smoked,True
2,Male,80.0,False,True,True,Private,Rural,105.92,32.5,never smoked,True
3,Female,49.0,False,False,True,Private,Urban,171.23,34.4,smokes,True
4,Female,79.0,True,False,True,Self-employed,Rural,174.12,24.0,never smoked,True
5,Male,81.0,False,False,True,Private,Urban,186.21,29.0,formerly smoked,True
6,Male,74.0,True,True,True,Private,Rural,70.09,27.4,never smoked,True
7,Female,69.0,False,False,False,Private,Urban,94.39,22.8,never smoked,True
8,Female,59.0,False,False,True,Private,Rural,76.15,NaN,Unknown,True
9,Female,78.0,False,False,True,Private,Urban,58.57,24.2,Unknown,True


In [10]:
# Create the virtual environment
environment = Environment(name="azure-env")
conda_env = CondaDependencies()

# Needed packages and scripts
conda_env.add_conda_package("pandas")
conda_env.add_conda_package("numpy")
conda_env.add_conda_package("scikit-learn")
conda_env.add_pip_package("azureml-defaults")

# Adding dependencies to the created environment
environment.python.conda_dependencies=conda_env

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# Define an early termination policy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

#TODO: Create the different params that you will be using during training
# Create different params that will be used during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0),
        '--max_iter': choice(10,25)
    }
)

#TODO: Create your estimator and hyperdrive config
# Create estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".",
    script='train.py',
    compute_target=amlcompute_cluster_name,
    environment=environment)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
    hyperparameter_sampling=param_sampling, 
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=24,
    max_concurrent_runs=4)

In [12]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_62a8ce62-0acf-443c-ae97-cd22b130210f
Web View: https://ml.azure.com/runs/HD_62a8ce62-0acf-443c-ae97-cd22b130210f?wsid=/subscriptions/bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8/resourcegroups/udacity-nano-degree/workspaces/udacity-courtlin&tid=3920f434-053b-4f53-aa54-b09a04e8f6dd

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-23T02:27:25.774885][API][INFO]Experiment created<END>\n""<START>[2021-09-23T02:27:26.351872][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-09-23T02:27:26.226963][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_62a8ce62-0acf-443c-ae97-cd22b130210f
Web View: https://ml.azure.com/runs/HD_62a8ce62-0acf-443c-ae97-cd22b130210f?wsid=/subscriptions/bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8/resourcegroups/udacity-nano-degree/workspaces/udacity-courtlin&tid=3920f434-053b-4f53-aa54-b09a04e8f6dd



{'runId': 'HD_62a8ce62-0acf-443c-ae97-cd22b130210f',
 'target': 'capstone',
 'status': 'Completed',
 'startTimeUtc': '2021-09-23T02:27:25.57923Z',
 'endTimeUtc': '2021-09-23T02:37:40.858681Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e5a1e17d-e90e-4188-8b54-92b358d79c33',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.9582484725050916',
  'best_child_run_id': 'HD_62a8ce62-0acf-443c-ae97-cd22b130210f_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://udacitycourtli3607057352.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_62a8ce62-0acf-443c-ae97-cd22b130210f/azureml-logs/hyperdrive.txt?sv=2019-07-0

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

AmlCompute is getting created. Consider calling wait_for_completion() first



In [14]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-ml,HD_62a8ce62-0acf-443c-ae97-cd22b130210f,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [51]:
# Get best model and diplay all details
best_run= run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
print('Best Run Accuracy:',best_run_metrics['Accuracy'])

['--C', '50', '--max_iter', '25']
['azureml-logs/55_azureml-execution-tvmps_31b6f5871a31b3e6b3541443a66b1f4f7a2a833d5e9c5b48d0048c37b52037a3_p.txt', 'azureml-logs/65_job_prep-tvmps_31b6f5871a31b3e6b3541443a66b1f4f7a2a833d5e9c5b48d0048c37b52037a3_p.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_31b6f5871a31b3e6b3541443a66b1f4f7a2a833d5e9c5b48d0048c37b52037a3_p.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/98_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.pkl']
Best Run Accuracy: 0.9582484725050916


In [50]:
# Retrieve and save best model
best_run = run.get_best_run_by_primary_metric()
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_31b6f5871a31b3e6b3541443a66b1f4f7a2a833d5e9c5b48d0048c37b52037a3_p.txt',
 'azureml-logs/65_job_prep-tvmps_31b6f5871a31b3e6b3541443a66b1f4f7a2a833d5e9c5b48d0048c37b52037a3_p.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_31b6f5871a31b3e6b3541443a66b1f4f7a2a833d5e9c5b48d0048c37b52037a3_p.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/98_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/model.pkl']

In [17]:
#TODO: Save the best model
# Save the best model
model=best_run.register_model(model_name='HYPERdrive-best-stroke-model', model_path='/outputs/model.pkl')
best_run.download_file('/outputs/model.pkl', 'HYPERdrive-best-stroke-model.pkl')

In [53]:
model

Model(workspace=Workspace.create(name='udacity-courtlin', subscription_id='bd6c48f0-b2f5-4fd8-b4de-3351b13cbee8', resource_group='udacity-nano-degree'), name=HYPERdrive-best-stroke-model, id=HYPERdrive-best-stroke-model:3, version=3, tags={}, properties={})

In [59]:
best_run.get_environment()

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210714.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "azure-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
    

TODO: In the cell below, print the logs of the web service and delete the service

In [27]:
compute_target.delete()